# Generating Piano Music With Transformer

[Original Colab here](https://colab.research.google.com/notebooks/magenta/piano_transformer/piano_transformer.ipynb#scrollTo=tciXVi5eWG_1)

## Imports and Set Up

In [21]:
print('Importing libraries...')

import numpy as np
import os
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

from google.colab import files

from tensor2tensor import models
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.utils import decoding
from tensor2tensor.utils import trainer_lib
from tensor2tensor import problems

import magenta.music as mm
from magenta.models.score2perf import score2perf

#@title Definitions
#@markdown Define a few constants and helper functions.

SF2_PATH = '/data/transformer/Yamaha-C5-Salamander-JNv5.1.sf2'
SAMPLE_RATE = 16000

# Upload a MIDI file and convert to NoteSequence.
def upload_midi():
  data = list(files.upload().values())
  if len(data) > 1:
    print('Multiple files uploaded; using only one.')
  return mm.midi_to_note_sequence(data[0])

# Decode a list of IDs.
def decode(ids, encoder):
  ids = list(ids)
  if text_encoder.EOS_ID in ids:
    ids = ids[:ids.index(text_encoder.EOS_ID)]
  return encoder.decode(ids)
  

    
    
    
    
    
#@title Setup and Load Checkpoint
#@markdown Set up generation from an unconditional Transformer
#@markdown model.

model_name = 'transformer'
hparams_set = 'transformer_tpu'
ckpt_path = '/data/transformer/checkpoints/unconditional_model_16.ckpt'

class PianoPerformanceLanguageModelProblem(score2perf.Score2PerfProblem):
  @property
  def add_eos_symbol(self):
    return True

problem = PianoPerformanceLanguageModelProblem()
unconditional_encoders = problem.get_feature_encoders()

# Set up HParams.
hparams = trainer_lib.create_hparams(hparams_set=hparams_set)
trainer_lib.add_problem_hparams(hparams, problem)
hparams.num_hidden_layers = 16
hparams.sampling_method = 'random'

# Set up decoding HParams.
decode_hparams = decoding.decode_hparams()
decode_hparams.alpha = 0.0
decode_hparams.beam_size = 1

# Create Estimator.
run_config = trainer_lib.create_run_config(hparams)
estimator = trainer_lib.create_estimator(
    model_name, hparams, run_config,
    decode_hparams=decode_hparams)

# Create input generator (so we can adjust priming and
# decode length on the fly).
def input_generator():
  global targets
  print(targets)
  global decode_length
  while True:
    print(targets)
    yield {
        'targets': np.array([targets], dtype=np.int32),
        'decode_length': np.array(decode_length, dtype=np.int32)
    }

# These values will be changed by subsequent cells.
targets = []
print(targets)
decode_length = 0
    
    
    
    
    
    
    
    

# Start the Estimator, loading from the specified checkpoint.
inputs = input_generator()
print('inputs', inputs)
input_fn = decoding.make_input_fn_from_generator(inputs)
print('input_fn', input_fn)
unconditional_samples = estimator.predict(
    input_fn, checkpoint_path=ckpt_path)

# "Burn" one.
_ = next(unconditional_samples)

#@title Generate from Scratch
#@markdown Generate a piano performance from scratch.
#@markdown
#@markdown This can take a minute or so depending on the length
#@markdown of the performance the model ends up generating.
#@markdown Because we use a 
#@markdown [representation](http://g.co/magenta/performance-rnn)
#@markdown where each event corresponds to a variable amount of
#@markdown time, the actual number of seconds generated may vary.

targets = []
decode_length = 1024
print('unconditional samples', unconditional_samples)
# Generate sample events.
sample_ids = next(unconditional_samples)['outputs']
# Decode to NoteSequence.
print('sample_ids', sample_ids)
print('encoder', unconditional_encoders['targets'])
midi_filename = decode(
    sample_ids,
    encoder=unconditional_encoders['targets'])
unconditional_ns = mm.midi_file_to_note_sequence(midi_filename)
print('midi filename', midi_filename)
# Play and plot.
mm.play_sequence(
    unconditional_ns,
    synth=mm.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
mm.plot_sequence(unconditional_ns)

#@title Download Performance as MIDI
#@markdown Download generated performance as MIDI (optional).
mm.sequence_proto_to_midi_file(
    unconditional_ns, './tmp/unconditional.mid')
# files.download('./tmp/unconditional.mid')

Importing libraries...
[]
inputs <generator object input_generator at 0x7f3a9657f7d8>
[]
[]
input_fn <function make_input_fn_from_generator.<locals>.input_fn at 0x7f3a96a3c7b8>
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'transformer/while/assert_greater/Assert/AssertGuard/Merge' type=Merge>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.6/dist-packages/tensor2tensor/models/transformer.py", line 1256, in inner_loop
    top_k)  File "/usr/local/lib/python3.6/dist-packages/tensor2tensor/layers/common_layers.py", line 2952, in sample_with_temperature
    tf.debugging.assert_greater(temperature, 0.0)  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/check_ops.py", line 966, in assert_greater
    y, data, summarize, message, name)  File "/usr/local/lib/python3.6/dist-packages/tensorflow_co

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'transformer/while/assert_greater/Assert/AssertGuard/Merge' type=Merge>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.6/dist-packages/tensor2tensor/models/transformer.py", line 1256, in inner_loop
    top_k)  File "/usr/local/lib/python3.6/dist-packages/tensor2tensor/layers/common_layers.py", line 2952, in sample_with_temperature
    tf.debugging.assert_greater(temperature, 0.0)  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/check_ops.py", line 966, in assert_greater
    y, data, summarize, message, name)  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/check_ops.py", line 371, in _binary_assert
    return control_flow_ops.Assert(condition, data, summarize=summarize)  File "/usr/local/lib/python3.6/dist-packages/t

unconditional samples <generator object Estimator.predict at 0x7f3a9657f150>
[]
sample_ids [224 286  33 ... 124 291  29]
encoder <magenta.models.score2perf.music_encoders.MidiPerformanceEncoder object at 0x7f3a96bf80b8>
midi filename /tmp/tmpx3mw7c_y_decode.mid


Loading BokehJS ...

In [16]:
print('Importing libraries...')

import numpy as np
import os
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

from google.colab import files

from tensor2tensor import models
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.utils import decoding
from tensor2tensor.utils import trainer_lib
from tensor2tensor import problems

import magenta.music as mm
from magenta.models.score2perf import score2perf

Importing libraries...


In [8]:
#@title Definitions
#@markdown Define a few constants and helper functions.

SF2_PATH = '/data/transformer/Yamaha-C5-Salamander-JNv5.1.sf2'
SAMPLE_RATE = 16000

# Upload a MIDI file and convert to NoteSequence.
def upload_midi():
  data = list(files.upload().values())
  if len(data) > 1:
    print('Multiple files uploaded; using only one.')
  return mm.midi_to_note_sequence(data[0])

# Decode a list of IDs.
def decode(ids, encoder):
  ids = list(ids)
  if text_encoder.EOS_ID in ids:
    ids = ids[:ids.index(text_encoder.EOS_ID)]
  return encoder.decode(ids)
  

In [17]:
#@title Setup and Load Checkpoint
#@markdown Set up generation from an unconditional Transformer
#@markdown model.

model_name = 'transformer'
hparams_set = 'transformer_tpu'
ckpt_path = '/data/transformer/checkpoints/unconditional_model_16.ckpt'

class PianoPerformanceLanguageModelProblem(score2perf.Score2PerfProblem):
  @property
  def add_eos_symbol(self):
    return True

problem = PianoPerformanceLanguageModelProblem()
unconditional_encoders = problem.get_feature_encoders()

# Set up HParams.
hparams = trainer_lib.create_hparams(hparams_set=hparams_set)
trainer_lib.add_problem_hparams(hparams, problem)
hparams.num_hidden_layers = 16
hparams.sampling_method = 'random'

# Set up decoding HParams.
decode_hparams = decoding.decode_hparams()
decode_hparams.alpha = 0.0
decode_hparams.beam_size = 1

# Create Estimator.
run_config = trainer_lib.create_run_config(hparams)
estimator = trainer_lib.create_estimator(
    model_name, hparams, run_config,
    decode_hparams=decode_hparams)

# Create input generator (so we can adjust priming and
# decode length on the fly).
def input_generator():
  global targets
  print(targets)
  global decode_length
  while True:
    print(targets)
    yield {
        'targets': np.array([targets], dtype=np.int32),
        'decode_length': np.array(decode_length, dtype=np.int32)
    }

# These values will be changed by subsequent cells.
targets = []
print(targets)
decode_length = 0


[]


In [19]:
# Start the Estimator, loading from the specified checkpoint.
inputs = input_generator()
print('inputs', inputs)
input_fn = decoding.make_input_fn_from_generator(inputs)
print('input_fn', input_fn)
unconditional_samples = estimator.predict(
    input_fn, checkpoint_path=ckpt_path)

# "Burn" one.
_ = next(unconditional_samples)

inputs <generator object input_generator at 0x7f3a9657f5c8>
[]
[]
input_fn <function make_input_fn_from_generator.<locals>.input_fn at 0x7f3acc39d840>
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'transformer/while/assert_greater/Assert/AssertGuard/Merge' type=Merge>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.6/dist-packages/tensor2tensor/models/transformer.py", line 1256, in inner_loop
    top_k)  File "/usr/local/lib/python3.6/dist-packages/tensor2tensor/layers/common_layers.py", line 2952, in sample_with_temperature
    tf.debugging.assert_greater(temperature, 0.0)  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/check_ops.py", line 966, in assert_greater
    y, data, summarize, message, name)  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/check_ops.py

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'transformer/while/assert_greater/Assert/AssertGuard/Merge' type=Merge>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.6/dist-packages/tensor2tensor/models/transformer.py", line 1256, in inner_loop
    top_k)  File "/usr/local/lib/python3.6/dist-packages/tensor2tensor/layers/common_layers.py", line 2952, in sample_with_temperature
    tf.debugging.assert_greater(temperature, 0.0)  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/check_ops.py", line 966, in assert_greater
    y, data, summarize, message, name)  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/check_ops.py", line 371, in _binary_assert
    return control_flow_ops.Assert(condition, data, summarize=summarize)  File "/usr/local/lib/python3.6/dist-packages/t

In [14]:
#@title Generate from Scratch
#@markdown Generate a piano performance from scratch.
#@markdown
#@markdown This can take a minute or so depending on the length
#@markdown of the performance the model ends up generating.
#@markdown Because we use a 
#@markdown [representation](http://g.co/magenta/performance-rnn)
#@markdown where each event corresponds to a variable amount of
#@markdown time, the actual number of seconds generated may vary.

targets = []
decode_length = 1024
print('unconditional samples', unconditional_samples)
# Generate sample events.
sample_ids = next(unconditional_samples)['outputs']
# Decode to NoteSequence.
print('sample_ids', sample_ids)
print('encoder', unconditional_encoders['targets'])
midi_filename = decode(
    sample_ids,
    encoder=unconditional_encoders['targets'])
unconditional_ns = mm.midi_file_to_note_sequence(midi_filename)
print('midi filename', midi_filename)
# Play and plot.
mm.play_sequence(
    unconditional_ns,
    synth=mm.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
mm.plot_sequence(unconditional_ns)

unconditional samples <generator object Estimator.predict at 0x7f3acc757a98>
[]
sample_ids [277 277 267 ... 180 144 145]
encoder <magenta.models.score2perf.music_encoders.MidiPerformanceEncoder object at 0x7f3acf042898>
midi filename /tmp/tmpqdojsv6h_decode.mid


Loading BokehJS ...

In [ ]:
#@title Download Performance as MIDI
#@markdown Download generated performance as MIDI (optional).
!mkdir -p ./tmp
mm.sequence_proto_to_midi_file(
    unconditional_ns, './tmp/unconditional.mid')
# files.download('./tmp/unconditional.mid')

In [26]:
!ls /data/transformer/primers

c_major_arpeggio.mid  clair_de_lune.mid  moonlight_sonata.mid
c_major_scale.mid     fur_elise.mid	 prelude_in_c_major.mid


In [32]:
#@title Choose Priming Sequence
#@markdown Here you can choose a priming sequence to be continued
#@markdown by the model.  We have provided a few, or you can
#@markdown upload your own MIDI file.
#@markdown
#@markdown Set `max_primer_seconds` below to trim the primer to a
#@markdown fixed number of seconds (this will have no effect if
#@markdown the primer is already shorter than `max_primer_seconds`).

def loadPrimer(filename):
    primer_ns = mm.midi_file_to_note_sequence(filename)

    # Handle sustain pedal in the primer.
    primer_ns = mm.apply_sustain_control_changes(primer_ns)

    # Trim to desired number of seconds.
    max_primer_seconds = 20  #@param {type:"slider", min:1, max:120}
    if primer_ns.total_time > max_primer_seconds:
      print('Primer is longer than %d seconds, truncating.' % max_primer_seconds)
      primer_ns = mm.extract_subsequence(
          primer_ns, 0, max_primer_seconds)

    # Remove drums from primer if present.
    if any(note.is_drum for note in primer_ns.notes):
      print('Primer contains drums; they will be removed.')
      notes = [note for note in primer_ns.notes if not note.is_drum]
      del primer_ns.notes[:]
      primer_ns.notes.extend(notes)

    # Set primer instrument and program.
    for note in primer_ns.notes:
      note.instrument = 1
      note.program = 0

    # Play and plot the primer.
    mm.play_sequence(
        primer_ns,
        synth=mm.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
    mm.plot_sequence(primer_ns)
    
    return primer_ns
    
filenames = {
    'C major arpeggio': '/data/transformer/primers/c_major_arpeggio.mid',
    'C major scale': '/data/transformer/primers/c_major_scale.mid',
    'Clair de Lune': '/data/transformer/primers/clair_de_lune.mid',
}    
primer_ns = loadPrimer('/data/transformer/primers/c_major_arpeggio.mid')

Loading BokehJS ...

In [37]:
def generateFromPrimer(primer_ns):
    #@title Generate Continuation
    #@markdown Continue a piano performance, starting with the
    #@markdown chosen priming sequence.

    targets = unconditional_encoders['targets'].encode_note_sequence(
        primer_ns)

    # Remove the end token from the encoded primer.
    targets = targets[:-1]

    decode_length = max(0, 4096 - len(targets))
    if len(targets) >= 4096:
      print('Primer has more events than maximum sequence length; nothing will be generated.')

    # Generate sample events.
    sample_ids = next(unconditional_samples)['outputs']

    # Decode to NoteSequence.
    midi_filename = decode(
        sample_ids,
        encoder=unconditional_encoders['targets'])
    ns = mm.midi_file_to_note_sequence(midi_filename)

    # Append continuation to primer.
    continuation_ns = mm.concatenate_sequences([primer_ns, ns])
    return continuation_ms

continuation_ms = generateFromPrimer(primer_ns)

# Play and plot.
mm.play_sequence(
    continuation_ns,
    synth=mm.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
mm.plot_sequence(continuation_ns)

[178, 294, 29, 190, 117, 295, 33, 189, 121, 36, 189, 124, 292, 41, 189, 129, 295, 45, 190, 133, 292, 48, 191, 136, 294, 53, 188, 141, 295, 57, 189, 145]


NameError: name 'continuation_ms' is not defined

In [36]:
#@title Download Continuation as MIDI
#@markdown Download performance (primer + generated continuation)
#@markdown as MIDI (optional).

mm.sequence_proto_to_midi_file(
    continuation_ns, '/tmp/continuation3.mid')
# files.download('/tmp/continuation.mid')

In [ ]:
#@title Setup and Load Checkpoint
#@markdown Set up generation from a melody-conditioned
#@markdown Transformer model.

model_name = 'transformer'
hparams_set = 'transformer_tpu'
ckpt_path = '/content/checkpoints/melody_conditioned_model_16.ckpt'

class MelodyToPianoPerformanceProblem(score2perf.AbsoluteMelody2PerfProblem):
  @property
  def add_eos_symbol(self):
    return True

problem = MelodyToPianoPerformanceProblem()
melody_conditioned_encoders = problem.get_feature_encoders()

# Set up HParams.
hparams = trainer_lib.create_hparams(hparams_set=hparams_set)
trainer_lib.add_problem_hparams(hparams, problem)
hparams.num_hidden_layers = 16
hparams.sampling_method = 'random'

# Set up decoding HParams.
decode_hparams = decoding.decode_hparams()
decode_hparams.alpha = 0.0
decode_hparams.beam_size = 1

# Create Estimator.
run_config = trainer_lib.create_run_config(hparams)
estimator = trainer_lib.create_estimator(
    model_name, hparams, run_config,
    decode_hparams=decode_hparams)

# These values will be changed by the following cell.
inputs = []
decode_length = 0

# Create input generator.
def input_generator():
  global inputs
  while True:
    yield {
        'inputs': np.array([[inputs]], dtype=np.int32),
        'targets': np.zeros([1, 0], dtype=np.int32),
        'decode_length': np.array(decode_length, dtype=np.int32)
    }

# Start the Estimator, loading from the specified checkpoint.
input_fn = decoding.make_input_fn_from_generator(input_generator())
melody_conditioned_samples = estimator.predict(
    input_fn, checkpoint_path=ckpt_path)

# "Burn" one.
_ = next(melody_conditioned_samples)

In [ ]:
#@title Choose Melody
#@markdown Here you can choose a melody to be accompanied by the
#@markdown model.  We have provided a few, or you can upload a
#@markdown MIDI file; if your MIDI file is polyphonic, the notes
#@markdown with highest pitch will be used as the melody.

# Tokens to insert between melody events.
event_padding = 2 * [mm.MELODY_NO_EVENT]

melodies = {
    'Mary Had a Little Lamb': [
        64, 62, 60, 62, 64, 64, 64, mm.MELODY_NO_EVENT,
        62, 62, 62, mm.MELODY_NO_EVENT,
        64, 67, 67, mm.MELODY_NO_EVENT,
        64, 62, 60, 62, 64, 64, 64, 64,
        62, 62, 64, 62, 60, mm.MELODY_NO_EVENT,
        mm.MELODY_NO_EVENT, mm.MELODY_NO_EVENT
    ],
    'Row Row Row Your Boat': [
        60, mm.MELODY_NO_EVENT, mm.MELODY_NO_EVENT,
        60, mm.MELODY_NO_EVENT, mm.MELODY_NO_EVENT,
        60, mm.MELODY_NO_EVENT, 62,
        64, mm.MELODY_NO_EVENT, mm.MELODY_NO_EVENT,
        64, mm.MELODY_NO_EVENT, 62,
        64, mm.MELODY_NO_EVENT, 65,
        67, mm.MELODY_NO_EVENT, mm.MELODY_NO_EVENT,
        mm.MELODY_NO_EVENT, mm.MELODY_NO_EVENT, mm.MELODY_NO_EVENT,
        72, 72, 72, 67, 67, 67, 64, 64, 64, 60, 60, 60,
        67, mm.MELODY_NO_EVENT, 65,
        64, mm.MELODY_NO_EVENT, 62,
        60, mm.MELODY_NO_EVENT, mm.MELODY_NO_EVENT,
        mm.MELODY_NO_EVENT, mm.MELODY_NO_EVENT, mm.MELODY_NO_EVENT
    ],
    'Twinkle Twinkle Little Star': [
        60, 60, 67, 67, 69, 69, 67, mm.MELODY_NO_EVENT,
        65, 65, 64, 64, 62, 62, 60, mm.MELODY_NO_EVENT,
        67, 67, 65, 65, 64, 64, 62, mm.MELODY_NO_EVENT,
        67, 67, 65, 65, 64, 64, 62, mm.MELODY_NO_EVENT,
        60, 60, 67, 67, 69, 69, 67, mm.MELODY_NO_EVENT,
        65, 65, 64, 64, 62, 62, 60, mm.MELODY_NO_EVENT        
    ]
}

melody = 'Twinkle Twinkle Little Star'  #@param ['Mary Had a Little Lamb', 'Row Row Row Your Boat', 'Twinkle Twinkle Little Star', 'Upload your own!']

if melody == 'Upload your own!':
  # Extract melody from user-uploaded MIDI file.
  melody_ns = upload_midi()
  melody_instrument = mm.infer_melody_for_sequence(melody_ns)
  notes = [note for note in melody_ns.notes
           if note.instrument == melody_instrument]
  del melody_ns.notes[:]
  melody_ns.notes.extend(
      sorted(notes, key=lambda note: note.start_time))
  for i in range(len(melody_ns.notes) - 1):
    melody_ns.notes[i].end_time = melody_ns.notes[i + 1].start_time
  inputs = melody_conditioned_encoders['inputs'].encode_note_sequence(
      melody_ns)
else:
  # Use one of the provided melodies.
  events = [event + 12 if event != mm.MELODY_NO_EVENT else event
            for e in melodies[melody]
            for event in [e] + event_padding]
  inputs = melody_conditioned_encoders['inputs'].encode(
      ' '.join(str(e) for e in events))
  melody_ns = mm.Melody(events).to_sequence(qpm=150)

# Play and plot the melody.
mm.play_sequence(
    melody_ns,
    synth=mm.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
mm.plot_sequence(melody_ns)


In [ ]:
#@title Generate Accompaniment for Melody
#@markdown Generate a piano performance consisting of the chosen
#@markdown melody plus accompaniment.

# Generate sample events.
decode_length = 4096
sample_ids = next(melody_conditioned_samples)['outputs']

# Decode to NoteSequence.
midi_filename = decode(
    sample_ids,
    encoder=melody_conditioned_encoders['targets'])
accompaniment_ns = mm.midi_file_to_note_sequence(midi_filename)

# Play and plot.
mm.play_sequence(
    accompaniment_ns,
    synth=mm.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
mm.plot_sequence(accompaniment_ns)

In [ ]:
#@title Download Accompaniment as MIDI
#@markdown Download accompaniment performance as MIDI (optional).

mm.sequence_proto_to_midi_file(
    accompaniment_ns, '/tmp/accompaniment.mid')
files.download('/tmp/accompaniment.mid')